#### **Import Libs**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import io
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from joblib import delayed, Parallel
from sklearn.datasets import load_digits


#### **Getting training && validation data**

In [2]:
table = pd.read_csv('./databases/dermatology.csv')

data = np.array(table)

data = data[:, 1:]

labels = []
for line in range(data.shape[0]):
  if(labels.count(data[line, data.shape[1]-1])==0):
    labels.append(data[line, data.shape[1]-1])

scaler = StandardScaler()

y = np.array(pd.get_dummies(data[:, data.shape[1]-1])).astype(np.float32)
X = (data[:, :(data.shape[1]-1)]).astype(np.float32)

scaler.fit(X) 

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25)
print('Conjuntos de treinamento e teste separados!')

Conjuntos de treinamento e teste separados!


#### **MLP**

In [3]:
digits = load_digits()
scaler = StandardScaler()
X = scaler.fit_transform(digits.data)
y = digits.target

train_size = int(0.8 * len(X))
train_X, test_X = X[:train_size], X[train_size:]
train_y, test_y = y[:train_size], y[train_size:]

# 2. Definir as possíveis configurações de rede neural em uma lista de dicionários
configs = [{'hidden_layer_sizes': (n_neurons,)*n_layers, 'max_iter': n_epochs} 
           for n_layers in [1, 2] 
           for n_neurons in [50, 100, 150, 200] 
           for n_epochs in [10, 30, 50, 70, 90, 110]]

# 3. Criar uma função que recebe uma configuração como argumento e retorna a acurácia da rede neural treinada com essa configuração
def train_neural_network(config):
    model = MLPClassifier(alpha=0.1, solver='sgd', learning_rate_init=0.01, momentum=0.9, learning_rate='adaptive', verbose=0, random_state=121, **config)
    model.fit(train_X, train_y)
    test_pred = model.predict(test_X)
    accuracy = accuracy_score(test_y, test_pred)
    return config, accuracy

# 4. Usar a função `Parallel` do módulo `joblib` para paralelizar o treinamento das redes neurais
results = Parallel(n_jobs=-1, verbose=10)(delayed(train_neural_network)(config) for config in configs)

# 5. Selecionar a configuração que teve a maior acurácia e imprimir seus resultados.
best_config, best_accuracy = max(results, key=lambda x: x[1])
print('Melhor configuração:', best_config)
print('Acurácia:', best_accuracy)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  38 out of  48 | elapsed:    6.8s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done  43 out of  48 | elapsed:    9.1s remaining:    1.0s


Melhor configuração: {'hidden_layer_sizes': (50,), 'max_iter': 70}
Acurácia: 0.9166666666666666


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   11.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   11.6s finished
